In [2]:
from zipfile import ZipFile

kmz = ZipFile('Functional Pumps.kmz', 'r')
kml = kmz.open('doc.kml', 'r').read()

In [ ]:
from lxml import html

doc = html.fromstring(kml)

# Znajdowanie dystansu od najbliższej studni

In [15]:
from geopy.distance import distance

def parseKMZFile(kmz_file_path: str) -> list:
    """
    Parses Google My Maps KMZ file into a list of pointmarks.
    
    Parameters
    ---
    - `kmz_file_path` - path to KMZ file to be analyzed.

    Returns
    ---
    A list of placemark coordinates in `[latitude, longitude]` format
    """

    kmz = ZipFile('Functional Pumps.kmz', 'r')
    kml = kmz.open('doc.kml', 'r').read()
    doc = html.fromstring(kml)

    wells_coordinates = []

    for placemark_coordinate in doc.cssselect("Document Placemark Point"):
        well_coordinates_raw: str = placemark_coordinate.text_content()
        well_coordinates_split = well_coordinates_raw.split(',')
        well_coordinates = (
        float(well_coordinates_split[1].strip()),
        float(well_coordinates_split[0].strip())
    )
        wells_coordinates.append(well_coordinates)
    return wells_coordinates

def closestWellFromLocation(origin_coordinates: float, wells_coordinates: list) -> tuple:
    """
    Returns the position of the well that is closest to origin point and distance between them.

    All coordinates should be in `[latitude, longitude]` format.
    """
    distances_to_wells = [
        distance(origin_coordinates, well_coordinates).m for well_coordinates in wells_coordinates
    ]
    minimal_distance = min(distances_to_wells)
    index_of_the_closest_well = distances_to_wells.index(minimal_distance)
    return(
        wells_coordinates[index_of_the_closest_well],
        minimal_distance
    )

In [17]:
random_point_near_wells = [5.74639, 19.12389]

wells_coordinates = parseKMZFile('Functional Pumps.kmz')

print(closestWellFromLocation(random_point_near_wells, wells_coordinates))

((5.734811, 19.129088), 1403.9564923898183)


In [19]:
closestWellFromLocation([5.74472, 19.13528], wells_coordinates)

((5.735001, 19.141562), 1280.3611485467688)